In [9]:
import numpy as np
import pandas as pd
from os.path import join as oj
import os
import pygsheets
import pandas as pd
import sys
import inspect
from datetime import datetime, timedelta

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
sys.path.append(parentdir + '/modeling')
import load_data
from fit_and_predict import fit_and_predict_ensemble
from functions import merge_data
from viz import  viz_interactive
import matplotlib.pyplot as plt
import update_modeling_results
import plotly.express as px
import plotly

# county-level plot

In [ ]:
print('loading data...')
NUM_DAYS_LIST = [1, 2, 3, 4, 5, 6, 7]
df_county = load_data.load_county_level(data_dir=oj(parentdir, 'data'))


num_days_in_past = 3
output_key = f'Predicted Deaths {num_days_in_past}-day'    
df_county = fit_and_predict_ensemble(df_county, 
                            outcome='deaths',
                            mode='eval_mode',
                            target_day=np.array([num_days_in_past]),
                            output_key=output_key)
df_county[output_key] = [v[0] for v in df_county[output_key].values]

In [10]:
update_modeling_results.predictions_plot(df_county, NUM_DAYS_LIST, num_days_in_past, output_key)

NameError: name 'output_key' is not defined

# hospital-level plot

In [ ]:
NUM_DAYS_LIST = [1, 2, 3, 4, 5, 6, 7]
df_county = load_data.load_county_level(data_dir=oj(parentdir, 'data'))
df_hospital = load_data.load_hospital_level(data_dir=oj(parentdir, 'data_hospital_level'))
df_county = add_preds(df_county, NUM_DAYS_LIST=NUM_DAYS_LIST, cached_dir=oj(parentdir, 'data')) # adds keys like "Predicted Deaths 1-day"
df = merge_data.merge_county_and_hosp(df_county, df_hospital)
df = add_severity_index(df, NUM_DAYS_LIST)
df = df.sort_values('Total Deaths Hospital', ascending=False)

In [ ]:
NUM_DAYS = len(NUM_DAYS_LIST)
np.repeat(range(NUM_DAYS), N) + 1